In [2]:
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
from scipy.interpolate.interpolate import interp1d
import plotly.graph_objects as go

In [1]:
in_folder = r'airfoil_data/'

In [85]:
df = pd.read_csv('data/0006_cd.csv')
data = df.melt(id_vars = ['x'], var_name = 'y', value_name = 'z').dropna().reset_index(drop=True)
data.y = data.y.astype(int)
data.dtypes

x    float64
y      int32
z    float64
dtype: object

In [4]:
# y = interp1d(df1.aoa, df1.cl, kind = 'cubic', fill_value='extrapolate')
# x_vals = np.linspace(-9, 21, 101)
# y(x_vals)
# df1 = pd.DataFrame({ 'aoa': x_vals, 'cl': y(x_vals)})
# df1['thickness'] = 12
# df5 = df5.sort_values(by='aoa')

In [109]:
def layout():
   camera = dict(eye=dict(x=0.1, y=-2, z=0.1))
   scene = dict(
      xaxis = dict(title='Cl'),
      yaxis = dict(title='Re'),
      zaxis = dict(title='Cd')
   )
   legend = dict(font = {'size': 12}, itemsizing = 'trace')
   return go.Layout(margin=dict(l=0, r=0, b=0, t=0), 
                     width=1024, height=768, 
                     scene=scene,
                     scene_camera=camera,
                     legend_title_text='Data',
                     legend=legend)

def plot2d():
        fig = go.Figure(layout=go.Layout(xaxis = dict(title='Cl'), yaxis = dict(title='Cd')))
        fig.add_trace(go.Scatter(x=df['x'], 
                                   y=df['3'],
                                   mode='lines',
                                   name='3'))
        fig.add_trace(go.Scatter(x=df['x'], 
                                   y=df['6'],
                                   mode='lines',
                                   name='6'))
        fig.add_trace(go.Scatter(x=df['x'], 
                                   y=df['9'],
                                   mode='lines',
                                   name='9'))
        fig.show()

def plot3d():
        fig = go.Figure(layout=layout())
        trace = data[data.y == 3]
        fig.add_trace(go.Scatter3d(x=trace.x, 
                                   y=trace.y,
                                   z=trace.z,
                                   mode='lines',
                                   name='3'))
        trace = data[data.y == 6]
        fig.add_trace(go.Scatter3d(x=trace.x, 
                                   y=trace.y,
                                   z=trace.z,
                                   mode='lines',
                                   name='6'))
        trace = data[data.y == 9]
        fig.add_trace(go.Scatter3d(x=trace.x, 
                                   y=trace.y,
                                   z=trace.z,
                                   mode='lines',
                                   name='9'))
        trace = mesh
        fig.add_trace(go.Scatter3d(x=trace.x, 
                                   y=trace.y,
                                   z=trace.z,
                                   mode='markers',
                                   name='mesh',
                                   marker=dict(size=3)))
        fig.show()

In [110]:
def interpolate_y(points, mesh, y_vals):
    """Get the interpolated x and z values at y_vals"""
    x = interp1d(points.y, points.x, kind='quadratic', fill_value='extrapolate')
    z = interp1d(points.y, points.z, kind='quadratic', fill_value='extrapolate')
    return mesh.append(pd.DataFrame({'x': x(y_vals), 'y': y_vals, 'z': z(y_vals)}), ignore_index=True)
    
def top_intercepts(relative_z, side):
    '''Points intercepting the same relative z for every curve'''
    points = pd.DataFrame()
    for y in data.y.unique():
        df = data[data.y == y]
        curve = df[df.x >= 0] if side == 'right' else df[df.x < 0]
        z = curve.z.min() + (curve.z.max() - curve.z.min()) * relative_z
        x = interp1d(curve.z, curve.x, kind = 'quadratic', fill_value='extrapolate')
        points = points.append({'x': x(z), 'y': y, 'z': z}, ignore_index=True)
    return points

def densing(y_vals):
    """Ratio-based mesh"""
    mesh = pd.DataFrame()
    z_rel = np.linspace(0, 1, 101)
    for z in z_rel:
        points = top_intercepts(z, side='right')
        mesh = interpolate_y(points, mesh, y_vals)
    for z in z_rel:
        points = top_intercepts(z, side='left')
        mesh = interpolate_y(points, mesh, y_vals)
    return mesh

In [112]:
y_vals = np.linspace(3, 9, 31)
mesh = densing(y_vals)
plot3d()

In [113]:
def create_curves():
    """Create a regular mesh based on the interpolated values"""
    curves = pd.DataFrame()
    x_vals = np.linspace(-1, 1, 201)
    x_vals = x_vals[ (mesh.x.min() < x_vals) & (x_vals < mesh.x.max()) ]
    for y in mesh.y.unique():
        points = mesh[mesh.y == y]
        z = interp1d(points.x, points.z, kind='quadratic', fill_value='extrapolate')
        curves = curves.append(pd.DataFrame({'x': x_vals, 'y': y, 'z': z(x_vals)}), ignore_index=True)
    return curves
        
def draw_curves():
    fig = go.Figure(layout=layout())
    fig.add_trace(go.Scatter3d(x=curves.x, 
                               y=curves.y,
                               z=curves.z,
                               mode='markers',
                               name='mesh',
                               marker=dict(size=3)))
    fig.show()

In [114]:
curves = create_curves()
draw_curves()

In [120]:
def draw_surface():
    """go.Surface requires a different data structure"""
    data = curves.pivot(index='y', columns='x', values='z')
    fig = go.Figure(go.Surface(
        z=data.values,
        x=data.columns,
        y=data.index,
        showscale=False,
        colorscale='algae',
        opacity=0.9,
        reversescale=True,
        hovertemplate=
        '<b>Cl</b>: %{x}<br>' +
        '<b>Re</b>: %{y}<br>' +
        '<b>Cd</b>: %{z}<extra></extra>'
    ), layout=layout())
    fig.show()

draw_surface()